In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 29.9 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#import libraries
import os
import pandas as pd
from time import time
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split, RandomSampler, SequentialSampler
import transformers
from transformers import BertConfig, BertForSequenceClassification, RobertaTokenizerFast, AutoModelForTokenClassification, AdamW, BertTokenizer, get_linear_schedule_with_warmup, AutoModel
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score

In [4]:
#download data 
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
print(len(df_train), '\n', df_train.head())
print(len(df_test), '\n', df_test.head())

34647 
    id                                       comment_text  toxicity
0   0                 fuck you you self righteous creep          3
1   1   stop stop the goddam vandalism or there ll be...         2
2   2  i agree rt does have a few shortcomings  but i...         0
3   3  if you would like verfiability here is the lin...         0
4   4  do you think there s consensus for me to be on...         0
9194 
       id                                       comment_text
0  34647  oh that great repository of free cultural work...
1  34648  my rfa  with apologies for the impersonal awb ...
2  34649  it looks like a number of articles you created...
3  34650  oh  but i see you ve been  block  for other  s...
4  34651  accord of the discussion in mariah carey compo...


## Preparing dataloader

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [46]:
df_train['comment_text'] = df_train['comment_text'].apply(lambda x: str(x).strip())
train_sentences = list(df_train.comment_text)
train_labels = list(df_train.toxicity)
df_test['comment_text'] = df_test['comment_text'].apply(lambda x: str(x).strip())
test_sentences = list(df_test.comment_text)

In [47]:
def get_max_len(sentences: list):
  max = 0
  sentence0 = ''
  for sentence in sentences:
    max = len(sentence) if len(sentence) > max else max
    sentence0 = sentence if len(sentence) >= max else sentence0
  return max, sentence0

MAX_LENGTH = 300
CLASSES_NUMBER = len(set(train_labels))

In [48]:
def tokenize(sentence: list, tokenizer, max_length):
    """
    input: one sentence ['some', 'sentence', 'here'];
    output: tokens_ids - tensor([1234, 23, 3241]), 
            attention_mask -  tensor([1, 1, 1, 1, 1, 1, 1, 1, 0,]),
            bert_tokens - ['what', 'both', 'al', 'pac', '##ino', 'and', 'robert', 'den'].
    """
    tokens_ids = []
    attention_mask = []
    bert_tokens = []
    tokenized_input = tokenizer(sentence,
                                add_special_tokens = True,
                                truncation = True,
                                max_length = max_length,           
                                pad_to_max_length = True,
                                return_attention_mask = True,   
                                return_tensors = 'pt',
                                is_split_into_words=False
                               )
    
    tokens_ids = tokenized_input['input_ids'][0]
    attention_mask = tokenized_input['attention_mask'][0]
    bert_tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
    return tokens_ids, attention_mask, bert_tokens

In [49]:
#tokenize 
# lists of bert tokens
bert_tokens = []
#tokens ids
tokens_ids = []
# attention masks
attentions_masks = []
# tokenize each sentence 
for sentence in train_sentences:
    token_id, attention_mask, bert_token = tokenize(sentence, tokenizer, MAX_LENGTH)
    tokens_ids.append(token_id)
    attentions_masks.append(attention_mask)
    bert_tokens.append(bert_token)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [50]:
#for test sentences 
#tokenize 
# lists of bert tokens
test_bert_tokens = []
#tokens ids
test_tokens_ids = []
# attention masks
test_attentions_masks = []
# tokenize each sentence 
for sentence in test_sentences:
    token_id, attention_mask, bert_token = tokenize(sentence, tokenizer, MAX_LENGTH)
    test_tokens_ids.append(token_id)
    test_attentions_masks.append(attention_mask)
    test_bert_tokens.append(bert_token)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [51]:
len(bert_tokens)

34646

In [52]:
len(train_labels)

34646

In [53]:
print("Bert tokens: ", bert_tokens[0])
print("Tokens ids: ", tokens_ids[0])
print("New labels: ", train_labels[0])
print("Attantion mask: ", attentions_masks[0])

Bert tokens:  ['[CLS]', 'fuck', 'you', 'you', 'self', 'righteous', 'creep', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

## Convert input_ids, attentions_masks and train_labels to TensorDataset

In [58]:
pt_input_ids = torch.stack(tokens_ids, dim=0)

pt_attention_masks = torch.stack(attentions_masks, dim=0)

pt_labels = torch.tensor(train_labels, dtype=torch.long)

In [59]:
#for test sentences
test_pt_input_ids = torch.stack(test_tokens_ids, dim=0)

test_pt_attention_masks = torch.stack(test_attentions_masks, dim=0)



In [61]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(pt_input_ids, pt_attention_masks, pt_labels)

# Create a 90-10 train-validation split.
train_size = int(0.999 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

34,611 training samples
   35 validation samples


In [62]:
#for test dataset
test_dataset = TensorDataset(test_pt_input_ids, test_pt_attention_masks)
# Divide the dataset by randomly selecting samples.

print('{:>5,} test samples'.format(len(test_dataset)))


9,194 test samples


## Convert TensorDataset to Dataloader

In [63]:
BATCH_SIZE = 16

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=BATCH_SIZE)

In [64]:
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=BATCH_SIZE)

In [65]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # Use the 124-layer, 1024-hidden, 16-heads, 340M parameters BERT model with an uncased vocab.
    num_labels = CLASSES_NUMBER, # The number of output labels--2 for binary classification. You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [66]:
# If there's a GPU available...

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.  
    
    device = torch.device('cuda')    


    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

model.to(device)

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Training

In [67]:
# Load the AdamW optimizer
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate 
                  eps = 1e-8 # args.adam_epsilon 
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [68]:
# Number of training epochs 
epochs = 2

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [69]:
import random

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):

    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    
    total_loss = 0

    model.train()
    
    for step, batch in enumerate(train_dataloader):
       
        if step % 100 == 0 and not step == 0:
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("  Average training loss: {0:.2f}".format(avg_train_loss))

======== Epoch 1 / 2 ========
Training...
  Batch   100  of  2,164.
  Batch   200  of  2,164.
  Batch   300  of  2,164.
  Batch   400  of  2,164.
  Batch   500  of  2,164.
  Batch   600  of  2,164.
  Batch   700  of  2,164.
  Batch   800  of  2,164.
  Batch   900  of  2,164.
  Batch 1,000  of  2,164.
  Batch 1,100  of  2,164.
  Batch 1,200  of  2,164.
  Batch 1,300  of  2,164.
  Batch 1,400  of  2,164.
  Batch 1,500  of  2,164.
  Batch 1,600  of  2,164.
  Batch 1,700  of  2,164.
  Batch 1,800  of  2,164.
  Batch 1,900  of  2,164.
  Batch 2,000  of  2,164.
  Batch 2,100  of  2,164.
  Average training loss: 0.77
======== Epoch 2 / 2 ========
Training...
  Batch   100  of  2,164.
  Batch   200  of  2,164.
  Batch   300  of  2,164.
  Batch   400  of  2,164.
  Batch   500  of  2,164.
  Batch   600  of  2,164.
  Batch   700  of  2,164.
  Batch   800  of  2,164.
  Batch   900  of  2,164.
  Batch 1,000  of  2,164.
  Batch 1,100  of  2,164.
  Batch 1,200  of  2,164.
  Batch 1,300  of  2,164.
  

In [70]:
# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions, true_labels = [], []

# Predict 
for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch) ##t.to(device)
  
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
  
    # Telling the model not to compute or store gradients, saving memory and 

    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

    DONE.


In [71]:
#Test dataset
# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions_test = []

# Predict 
for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch) ##t.to(device)
  
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
  
    # Telling the model not to compute or store gradients, saving memory and 

    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    #label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions_test.append(logits)
    #true_labels.append(label_ids)

print('    DONE.')

    DONE.


In [72]:
submission = pd.read_csv('submission.csv')
submission.head()

,id,prediction
0,34647,0
1,34648,0
2,34649,0
3,34650,0
4,34651,0


In [73]:
predictions_test_0 = np.concatenate(predictions_test, axis=0)
predictions_test_0

array([[ 5.7892346 ,  2.9139578 , -0.7423075 , -2.3402362 , -3.460374  ,
        -2.8209934 ],
       [ 7.370486  ,  0.5536067 , -1.7443788 , -2.4937348 , -2.5184624 ,
        -1.96266   ],
       [ 7.213716  , -0.06232698, -1.8447533 , -2.3940964 , -2.1407135 ,
        -1.3780146 ],
       ...,
       [ 1.3675389 ,  3.7210956 ,  1.6388266 , -0.774359  , -2.7443225 ,
        -4.286558  ],
       [-2.3002644 ,  0.6463383 ,  0.68021655,  1.6560824 ,  0.9153693 ,
        -1.596381  ],
       [-0.12701559,  2.6128974 ,  2.931504  ,  0.5130284 , -2.853128  ,
        -4.496347  ]], dtype=float32)

In [74]:
all_predictions_test = np.argmax(predictions_test_0, axis=1)
len(all_predictions_test)

9194

In [75]:
all_predictions_test[-1]

2

In [76]:
submission['prediction'] = all_predictions_test.tolist()

In [77]:
submission

,id,prediction
0,34647,0
1,34648,0
2,34649,0
3,34650,2
4,34651,1
...,...,...
9189,43836,4
9190,43837,2
9191,43838,1
9192,43839,3


In [78]:
all_predictions = np.concatenate(predictions, axis=0)
all_true_labels = np.concatenate(true_labels, axis=0)

In [79]:
predict = np.argmax(all_predictions, axis=1)

In [80]:
true = all_true_labels

In [81]:
from sklearn.metrics import f1_score

f1 = f1_score(predict, true, average='micro') 

print ("F1 score: {:.2%}".format(f1))

F1 score: 74.29%


In [82]:
submission.to_csv('submission_test.csv', index=False)

In [83]:
ff = pd.read_csv('submission_test.csv')

In [84]:
ff

,id,prediction
0,34647,0
1,34648,0
2,34649,0
3,34650,2
4,34651,1
...,...,...
9189,43836,4
9190,43837,2
9191,43838,1
9192,43839,3
